# Simulating alternative skipped exon experiments


### Simulation model





Here, each file is a different RBP motif as a position frequency matrix.  So, the first step is to compile all of these files into a suitable database.  In particular, we can parse each motifs (position frequency matrix) from each file in motifpath (downloaded top10align_motifs folder), create a database (list of arrays), and save as binary format (motif.list):

In [1]:
import os.path
import pandas as pd
import numpy as np
from six.moves import cPickle
import matplotlib.pyplot as plt
%matplotlib inline

#### Load a motif database for drosophila melanogaster

The data comes from Ray et al. "A compendium of RNA-binding motifs for decoding gene regulation" (http://www.nature.com/nature/journal/v499/n7457/abs/nature12311.html). The link to the motifs I downloaded is here: 

\$ wget http://hugheslab.ccbr.utoronto.ca/supplementary-data/RNAcompete_eukarya/top10align_motifs.tar.gz

\$ tar xzvf top10align_motifs.tar.gz

In [ ]:
# load motifs
motiflist = 'motif.pickle'  
if os.path.isfile(motiflist):

    # load motif list from file
    f = open(motiflist, 'rb')
    motif_set = cPickle.load(f)
    f.close()

else:
    # download motifs
    motifpath = 'top10align_motifs/'   # directory where motif files are located

    # get all motif files in motifpath directory
    listdir = os.listdir(motifpath)

    # parse motifs
    motif_set = []
    for files in listdir:
        df = pd.read_table(os.path.join(motifpath,files))
        motif_set.append(df.iloc[0::,1::].transpose())

    # save motifs    
    f = open(motiflist, 'wb')
    cPickle.dump(motif_set, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

## Simulation procedure

### Setup motif model for each alternative exon event

In [2]:
# setup motif code 
max_motifs_exon = 7
num_models = 10      # number of regulatory grammars
distance_rate = 10
interaction_rate = 10
distance_offset = 3
motif_range = [0, 40]
exon_size = 100
intron_size = 150
region_size = np.array([exon_size, intron_size, 
                 intron_size, exon_size, 
                 exon_size, intron_size, 
                 intron_size, exon_size])

motif_model = []
for i in range(num_models):
    exon_model = []
    for k in range(8):
        status = True
        while status:
            # simulate number of motifs for each region
            num_motifs = np.ceil(np.random.randint(1, max_motifs_exon)).astype(int)

            # simulate position offset from border
            start_offset = np.ceil(np.random.exponential(scale=distance_rate)).astype(int) + distance_offset

            # exponential separation rate between motifs
            distance_scale = (region_size[k]-start_offset)/num_motifs/10

            # simulate distance between motifs 
            separation = np.ceil(np.random.exponential(scale=interaction_rate, size=num_motifs)) + distance_offset

            # randomly select motifs from top10align
            motif_index = np.random.randint(motif_range[0], motif_range[1], size=num_motifs)

            # build position weight matrix model
            if (k%2) == 0:
                reverse = True
            else:
                reverse = False
                
            pwm = np.ones((4, start_offset))/4
            for j in range(num_motifs):
                if reverse:
                    pwm = np.hstack([np.ones((4,separation[j]))/4, motif_set[motif_index[j]], pwm])
                else:
                    pwm = np.hstack([pwm, motif_set[motif_index[j]], np.ones((4,separation[j]))/4])
            remainder = region_size[k]-pwm.shape[1]
            if remainder > 0:
                if reverse:
                    pwm = np.hstack((np.ones((4,remainder))/4, pwm))
                else:
                    pwm = np.hstack((pwm, np.ones((4,remainder))/4))
                status = False

        model = {'pwm': pwm,
                 'num_motifs': num_motifs, 
                 'start_offset': start_offset, 
                 'distance_scale': distance_scale, 
                 'separation': separation, 
                 'motif_index': motif_index}
        exon_model.append(model)
    motif_model.append(exon_model)
        

### Create gene expression model

In [3]:
# setup cell type gene expression profiles
num_cell_types = 200
num_genes = 5000

# random gene states
on_state = 4
on_std = 1.5
off_state = 0
off_std = 1
pop_frac = .85

gene_expression_model = []
for i in range(num_cell_types):
    gene_state = np.random.binomial(1, pop_frac, num_genes)

    on_index = np.where(gene_state==1)[0]
    off_index = np.where(gene_state==0)[0]
    num_on = len(on_index)
    num_off = len(off_index)
    
    Z_off = np.random.normal(off_state, off_std, num_off)
    Z_on = np.random.normal(on_state, on_std, num_on)

    Z = np.zeros((num_genes))
    Z[on_index] = Z_on
    Z[off_index] = Z_off

    model = {'gene_expression': Z,
             'gene_state': gene_state,
             'on_index': on_index,
             'off_index': off_index,
             'num_off': num_off,
             'num_on': num_on
             }
    gene_expression_model.append(model)

### Setup interaction model

Get a list of unique motifs within alternative exon model

In [5]:
# get list of unique motifs
motif_index = []
for exon_model in motif_model:
    for model in exon_model:
        motif_index.append(model['motif_index'])
        
motifs = np.hstack(motif_index)
unique_motifs = np.unique(motifs)
num_unique = len(unique_motifs)


Count number of motifs for each exon model

In [20]:
# get motif count for each exon model
motif_count = np.zeros((num_models, num_unique))
for i in range(num_models):
    exon_model = motif_model[i]
    for model in exon_model:
        motif_count[i, unique_motifs[model['motif_index']]] += 1

Associate each RBP's with multiple motifs

In [21]:
# setup rbp interaction code - associate rbps to multiple motifs
num_rbp = 100
interaction_rate = 6
rbp_code = []
for i in range(num_rbp):
    num_interactions = min(np.ceil(np.random.exponential(interaction_rate)).astype(int), num_unique)
    rbp_interaction = np.random.permutation(range(num_unique))[:num_interactions]
    interaction_state = np.random.randint(0,2,num_interactions)
    model = {'rbp_interaction': rbp_interaction,
             'num_interactions': num_interactions,
             'interaction_state': interaction_state,
             }
    rbp_code.append(model)
    

Reverse of the previous step: get a catalogue of each RBP interaction for each motif

In [14]:
# get catalogue of rbp interactions with each motif
motif_interactions = []
for motif in unique_motifs:
    
    # loop through each rbp to see if it regulates the motif
    rbp_players = []
    for i in range(num_rbp):
        code = rbp_code[i]
        rbp_interaction = code['rbp_interaction'] # <-- list of motifs that rbp interacts with
        match_index = np.where(motif == rbp_interaction)[0] # <-- find which rbp regulate current motif

        if len(match_index) > 0: 
            # rbp interaction type (enhancer or silencer)
            interaction_type = code['interaction_state'][match_index] 
            rbp_players.append([i, interaction_type])
    motif_interactions.append(np.array(rbp_players))

Determine percentage spliced in based on gene expression levels of RBP, which motifs are present, and the state that the RBP works on the given motif

In [101]:
# figure out psi for each motif_model based on gene_expression of each rbp and their effect on state
psi = np.zeros((num_cell_types, num_models))
for i in range(num_cell_types):
    gene_expression = gene_expression_model[i]['gene_expression']

    for j in range(num_models):
        motifs = unique_motifs[motif_count[j] > 1]
        count = motif_count[j][motif_count[j] > 1]

        rbp = []
        state = []
        for motif in motifs:
            rbp.append(motif_interactions[motif][:,0])
            state.append(motif_interactions[motif][:,1])

        rbp = np.hstack(rbp)
        state = np.hstack(state)
        tpm = np.exp(gene_expression[rbp])
        tpm /= sum(tpm)
        psi[i,j] =sum(tpm*state)


Convert the position weight matrices into a realized simulated sequence

In [1]:
def simulate_sequences(pwm, num_sequenes):
    cum_pwm = pwm.cumsum(axis=0)
    sequence = []
    for i in range(num_sequences):
        Z = np.random.uniform(0, 1, cum_pwm.shape[1])
        sequence.append(np.argmin((cum_pwm - Z)**2, axis=0))
    return sequence

def simulate_skipped_exon_sequences(exon_model, num_sequences):
    pwm_up_exon = np.hstack([exon_model[0]['pwm'], exon_model[1]['pwm']])
    pwm_ae_exon_1 = np.hstack([exon_model[1]['pwm'], exon_model[2]['pwm']])
    pwm_ae_exon_2 = np.hstack([exon_model[2]['pwm'], exon_model[3]['pwm']])
    pwm_down_exon = np.hstack([exon_model[3]['pwm'], exon_model[4]['pwm']])

    up_exon_seq = simulate_sequences(pwm_up_exon, num_sequenes)
    ae_exon_seq_1 = simulate_sequences(pwm_ae_exon_1, num_sequenes)
    ae_exon_seq_2 = simulate_sequences(pwm_ae_exon_2, num_sequenes)
    down_exon_seq = simulate_sequences(pwm_down_exon, num_sequenes)

    return up_exon_seq, ae_exon_seq_1, ae_exon_seq_2, down_exon_seq

up_exon_seq, ae_exon_seq_1, ae_exon_seq_2, down_exon_seq = 
                    simulate_skipped_exon_sequences(exon_model, num_sequences)

### Split data and save

In [ ]:
def convert_one_hot(seq):
    """convert a sequence into a 1-hot representation"""
    
    nucleotide = 'ACGU'
    N = len(seq)
    one_hot_seq = np.zeros((4,N))
    for i in xrange(200):         
        #for j in range(4):
        #    if seq[i] == nucleotide[j]:
        #        one_hot_seq[j,i] = 1
        index = [j for j in xrange(4) if seq[i] == nucleotide[j]]
        one_hot_seq[index,i] = 1
        
    return one_hot_seq


def subset_data(data, label, sub_index):
    """returns a subset of the data and labels based on sub_index"""
    
    num_labels = len(np.unique(label))
    num_sub = len(sub_index)
    
    sub_set = np.zeros((num_sub, 4, len(data[0])))
    sub_set_label = np.zeros((num_sub, num_labels))
    
    k = 0;
    for index in sub_index:
        sub_set[k] = convert_one_hot(data[index])
        sub_set_label[k,label[index]] = 1
        k += 1

    sub_set_label = sub_set_label.astype(np.uint8)
    
    return (sub_set, sub_set_label)

def split_data(data, label, split_size):
    """split data into train set, cross-validation set, and test set"""
    
    # number of labels
    num_labels = len(np.unique(label))

    # determine indices of each dataset
    N = len(data)
    cum_index = np.cumsum(np.multiply([0, split_size[0], split_size[1], split_size[2]],N)).astype(int) 

    # shuffle data
    shuffle = np.random.permutation(N)

    # training dataset
    train_index = shuffle[range(cum_index[0], cum_index[1])]
    cross_validation_index = shuffle[range(cum_index[1], cum_index[2])]
    test_index = shuffle[range(cum_index[2], cum_index[3])]

    # create subsets of data based on indices 
    train = subset_data(data, label, train_index)
    cross_validation = subset_data(data, label, cross_validation_index)
    test = subset_data(data, label, test_index)
    
    return train, cross_validation, test

# percentage for each dataset
train_size = 0.7
cross_validation_size = 0.15
test_size = 0.15


# get indices for each dataset
print "Splitting dataset into train, cross-validation, and test"
split_size = [train_size, cross_validation_size, test_size]
train, cross_validation, test = split_data(data, label, split_size)

# save training dataset in one-hot representation
print "Saving dataset"
f = open(filename+'_data.pickle', 'wb')
cPickle.dump(train, f, protocol=cPickle.HIGHEST_PROTOCOL)
cPickle.dump(cross_validation, f, protocol=cPickle.HIGHEST_PROTOCOL)
cPickle.dump(test, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

# save training dataset in one-hot representation
print "Saving model"
f = open(filename+'_model.pickle', 'wb')
cPickle.dump(options, f, protocol=cPickle.HIGHEST_PROTOCOL)
cPickle.dump(model, f, protocol=cPickle.HIGHEST_PROTOCOL)
cPickle.dump(seq_model, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()


Now, we are ready to build and test our deep learning model.

In [ ]:
from six.moves import cPickle
import numpy as np
import sys

# load training data
filename = 'train_data_10000_200_10_20.pickle'
f = open(filename, 'rb')
train = cPickle.load(f)
cross_validation = cPickle.load(f)
test = cPickle.load(f)
f.close()

# separate data
train_set, train_set_label = train
cross_validation_set, cross_validation_set_label = cross_validation
test_set, test_set_label = test

# munge data for deep learning
train_set = munge_data(train_set)
cross_validation_set = munge_data(cross_validation_set)
test_set = munge_data(test_set)

    

In [ ]:
import os

outdir = 'data'
if not os.path.isdir(outdir):
    os.mkdir(outdir)
    print "making directory: " + outdir

In [ ]:
ls

In [ ]:
from six.moves import cPickle

# load motif list from file
filename = 'model_10000_200_10_20.pickle'
f = open(filename, 'rb')
model = cPickle.load(f)
options = cPickle.load(f)
seq_model = cPickle.load(f)
f.close()

# options = [motif_set, M, G, interaction_rate, distance_scale, offset, maxMotif]
# model [motifs, grammar, distance]
# seq_model
